# Epic 7: Data Processing and API Logic for Threatened Plant Index Visualization

Name: Zihan

### 步骤一：加载与预处理数据 📂

此单元格负责我们后端服务的**数据基础**。

1.  **导入库**: 我们导入了 `pandas` 用于常规数据处理，`geopandas` 用于处理地理空间数据，以及 `shapely.wkt` 这一关键工具。
2.  **加载数据**:
    * `Table14_TSX_Table_VIC_version4.csv` 被加载到一个标准的 pandas DataFrame (`df_tsx`) 中，它包含了所有州随年份变化的指标。
    * `Table15_StateShapeTable.csv` 也被加载进来，它包含了绘制地图所需的州边界信息。
3.  **核心转换**: 最关键的一步是 `gpd.GeoDataFrame(...)`。它使用 `shapely.wkt.loads` 将 CSV 文件中纯文本格式的 `MULTIPOLYGON` 字符串，转换成 `geopandas` 可以理解和绘制的**实际几何对象**。这样，一个普通的 DataFrame 就升级成了一个强大的 `GeoDataFrame` (`gdf_states`)。
4.  **验证**: 最后的 `print` 语句用于检查数据是否正确加载，并让我们注意到 `TSX数据` 和 `Shape数据` 中包含的州列表略有不同，这有助于我们了解数据的覆盖范围。

In [3]:
import pandas as pd
import geopandas as gpd
from shapely import wkt # 用于将字符串转换回几何对象

# --- 1. 加载和预处理数据 ---

# 加载时间序列数据
df_tsx = pd.read_csv('Table14_TSX_Table_VIC_version4.csv')

# 加载地理形状数据
df_shapes = pd.read_csv('Table15_StateShapeTable.csv')

# 将 CSV 中的 WKT (Well-Known Text) 字符串转换为真实的几何对象
# 这是关键一步，让 pandas DataFrame 变成 GeoDataFrame
df_shapes['geometry'] = df_shapes['geometry'].apply(wkt.loads)
gdf_states = gpd.GeoDataFrame(df_shapes, geometry='geometry')

# 我们可以检查一下数据
print("TSX Data Head:")
display(df_tsx.head())
print("\nGeoDataFrame Head:")
display(gdf_states.head())
print(f"\nAvailable states in TSX data: {df_tsx['state'].unique()}")
print(f"Available states in Shape data: {gdf_states['state'].unique()}")

TSX Data Head:


,year,index_value,value_type,annual_mean_temp,annual_precip_sum,annual_radiation_sum,state
0,2000,1.000000,Historical,14.757990,615.8,5848.0400,Victoria
1,2001,0.851265,Historical,14.526712,564.2,5662.2200,Victoria
2,2002,0.738937,Historical,14.698013,432.4,5789.7603,Victoria
3,2003,0.728083,Historical,14.443562,571.0,5895.1000,Victoria
4,2004,0.647233,Historical,14.277344,621.2,5806.9700,Victoria



GeoDataFrame Head:


,state,geometry
0,New South Wales,"MULTIPOLYGON (((-31.509 159.062, -31.509 159.0..."
1,Victoria,"MULTIPOLYGON (((-39.158 146.293, -39.157 146.2..."
2,Queensland,"MULTIPOLYGON (((-10.683 142.531, -10.682 142.5..."
3,South Australia,"MULTIPOLYGON (((-38.063 140.66, -38.062 140.66..."
4,Western Australia,"MULTIPOLYGON (((-35.191 117.87, -35.191 117.87..."



Available states in TSX data: ['Victoria' 'New South Wales' 'South Australia' 'Western Australia'
 'Australian Capital Territory' 'National']
Available states in Shape data: ['New South Wales' 'Victoria' 'Queensland' 'South Australia'
 'Western Australia' 'Tasmania' 'Northern Territory'
 'Australian Capital Territory' 'Other Territories']


### 步骤二：定义后端核心功能函数 🛠️

在这个单元格中，我们定义了三个核心函数，它们**模拟了未来将要构建的 Web API 的功能**。每个函数都为一个特定的前端需求服务，负责从原始数据中提取并格式化所需的信息。

1.  `get_map_base_geojson()`:
    * **目的**: 为前端提供绘制澳大利亚地图所需的**基础地理轮廓**。
    * **工作**: 直接将 `gdf_states` 这个 `GeoDataFrame` 转换为 **GeoJSON** 格式。这是一种标准的地理信息数据格式，前端的地图库（如 Leaflet, Mapbox）可以直接使用。
    * **特点**: 这个数据是静态的，前端在加载页面时只需请求一次。

2.  `get_choropleth_data_for_year(year)`:
    * **目的**: 根据用户选择的**年份**，提供用于渲染 Choropleth Map (分区统计地图) 颜色的**数据**。
    * **工作**: 接收一个年份，筛选出 `df_tsx` 中该年份的所有数据，并将其格式化为一个简单的 Python 字典 `{'州名': 'TSX指数'}`。
    * **特点**: 返回的数据包非常**轻量**，适合在用户与时间滑块交互时频繁请求。

3.  `get_state_timeseries_data(state)`:
    * **目的**: 根据用户点击的**州**，提供绘制右侧**折线图**所需的完整时间序列数据。
    * **工作**: 接收一个州名，筛选出 `df_tsx` 中该州从 2000 年到 2027 年的**所有相关数据**（TSX 指数、天气变量、数据类型等）。
    * **特点**: 返回的数据格式为 JSON 记录数组 (`[{...}, {...}]`)，前端可以轻松地遍历这个数组来绘制图表。

In [4]:
# --- 2. 定义后端需要提供的核心功能函数 ---

def get_map_base_geojson():
    """
    功能：提供所有州的地理边界数据（GeoJSON格式）。
    用途：前端初始化时调用一次，用于绘制地图基本轮廓。
    返回：一个GeoJSON格式的字符串。
    """
    # to_json() 会自动将 GeoDataFrame 转换为 GeoJSON
    return gdf_states.to_json()

def get_choropleth_data_for_year(year: int):
    """
    功能：根据指定年份，获取所有州的 TSX 指数。
    用途：当用户在地图上滑动时间滑块时，前端调用此函数获取该年份的数据来更新地图颜色。
    参数：year - 用户选择的年份 (e.g., 2010)
    返回：一个字典，键是州名，值是该州的 TSX 指数。 e.g., {'Victoria': 0.45, ...}
    """
    if year not in df_tsx['year'].unique():
        return {"error": f"Year {year} not found in data."}
        
    # 筛选出指定年份的数据
    data_for_year = df_tsx[df_tsx['year'] == year]
    
    # 将结果转换为 {state: index_value} 的字典格式，方便前端使用
    result = pd.Series(data_for_year.index_value.values, index=data_for_year.state).to_dict()
    return result

def get_state_timeseries_data(state: str):
    """
    功能：获取指定州的完整时间序列数据（TSX 和所有天气变量）。
    用途：当用户在地图上点击某个州时，前端调用此函数获取数据来绘制右侧的折线图。
    参数：state - 用户选择的州名 (e.g., "Victoria")
    返回：一个包含该州所有数据的JSON对象（记录列表格式）。
    """
    if state not in df_tsx['state'].unique():
        return {"error": f"State '{state}' not found in data."}

    # 筛选出指定州的数据
    state_data = df_tsx[df_tsx['state'] == state].copy()
    
    # 为了方便前端处理，我们可以直接将筛选后的 DataFrame 转为 JSON
    # 'records' 格式会生成一个列表，每个元素是一个字典，非常适合前端渲染
    # e.g., [{'year': 2000, 'index_value': 1.0, ...}, {'year': 2001, ...}]
    return state_data.to_json(orient='records')

### 步骤三：在 Notebook 中测试函数 ✅

这个单元格的目的是**验证我们在步骤二中定义的函数是否能正常工作**，并产出我们期望格式的数据。在将逻辑迁移到真实的 Web 服务器之前进行这一步，可以大大简化调试过程。

1.  **测试 GeoJSON**: 我们调用 `get_map_base_geojson()` 并打印其输出的前200个字符。目的是确认它成功生成了一个以 `{"type": "FeatureCollection", ...}` 开头的有效 GeoJSON 字符串。
2.  **测试 Choropleth 数据**: 我们以 `2010` 年为例，调用 `get_choropleth_data_for_year()`。输出结果是一个清晰的字典，键是州名，值是该州在2010年的 TSX 指数，格式完全正确。
3.  **测试时间序列数据**: 我们分别以 `'Victoria'` 和 `'National'` 为例，调用 `get_state_timeseries_data()`。输出结果是一个 JSON 数组，其中每个对象都代表了一年的完整数据。这证明了函数能够正确地按州筛选并格式化数据。

由于所有测试都返回了预期的结果，我们可以确信这些核心逻辑是**正确且可靠的**，已经为下一步集成到 Flask Web 应用中做好了准备。

In [5]:
# --- 3. 在 Notebook 中测试这些函数 ---

print("\n--- Testing Functions ---")

# 测试1: 获取地图的 GeoJSON
print("\n[Test] GeoJSON for map base:")
geojson_data = get_map_base_geojson()
# GeoJSON会很长，我们只打印前200个字符看看格式对不对
print(geojson_data[:200] + "...") 

# 测试2: 获取2010年的 Choropleth 数据
print("\n[Test] Choropleth data for year 2010:")
choropleth_data_2010 = get_choropleth_data_for_year(2010)
print(choropleth_data_2010)

# 测试3: 获取 Victoria 州的折线图数据
print("\n[Test] Timeseries data for Victoria:")
victoria_timeseries = get_state_timeseries_data('Victoria')
print(victoria_timeseries)

# 测试4: 获取 National 的默认数据
# 假设你的数据中有 'National' 这一项
# 如果没有，你需要创建 'National' 的数据（比如所有州的平均值）
# 这里我们假设数据文件里已经包含了 'National'
print("\n[Test] Timeseries data for National:")
national_timeseries = get_state_timeseries_data('National')
print(national_timeseries)


--- Testing Functions ---

[Test] GeoJSON for map base:
{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {"state": "New South Wales"}, "geometry": {"type": "MultiPolygon", "coordinates": [[[[-31.508856162538123, 159.0...

[Test] Choropleth data for year 2010:
{'Victoria': 0.454594001858494, 'New South Wales': 0.433957342449423, 'South Australia': 0.971380029575459, 'Western Australia': 0.438504885066282, 'Australian Capital Territory': 1.15055990457538, 'National': 0.552198806798463}

[Test] Timeseries data for Victoria:
[{"year":2000,"index_value":1.0,"value_type":"Historical","annual_mean_temp":14.75799,"annual_precip_sum":615.8,"annual_radiation_sum":5848.04,"state":"Victoria"},{"year":2001,"index_value":0.8512650869,"value_type":"Historical","annual_mean_temp":14.526712,"annual_precip_sum":564.2,"annual_radiation_sum":5662.22,"state":"Victoria"},{"year":2002,"index_value":0.7389365802,"value_type":"Historical","annual_mean_temp":14.698013,"an